## 0. Import Libraries

In [19]:
from datetime import datetime, timedelta
from glob import glob

import numpy as np
import pandas as pd
import pyproj
import rioxarray
import salem
import xarray as xr
from shapely.geometry import mapping

import cartopy
import matplotlib.pyplot as plt
import proplot
from matplotlib.colors import BoundaryNorm, ListedColormap
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
from dea_tools.spatial import xr_vectorize, xr_rasterize
from wrf import ll_to_xy
import matplotlib.patheffects as pe
from netCDF4 import Dataset
from SPAEF_metric import SPAEF
from pyproj import CRS
from rasterio.enums import Resampling

import warnings
warnings.filterwarnings('ignore')

## 1. Simulation Data

In [3]:
variable = 'PRCP'
general_path = 'data'

# ensemble members
micro_options = [
    'LIN',
    'THOMPSON',
    'WSM6'
]

luse_options = [
    'urban',
    'nourban'
]

### 1.1 ERA5

#### 1.1.1 Data Options

In [4]:
run_data = 'era5'

# data path
data_path = glob(fr'{general_path}/*{run_data}**{variable}*')

In [5]:
data_path

['data/era5_LIN_nourban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/era5_LIN_urban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/era5_THOMPSON_nourban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/era5_THOMPSON_urban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/era5_WSM6_nourban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/era5_WSM6_urban_PRCP_wrfout_d03_2017-07-18.nc']

#### 1.1.2 Open Data

In [6]:
# open data
dt = xr.open_mfdataset(data_path,
                       concat_dim='ens')[variable]

# instead of 201 grid we want 200 grid in each direction
dt = dt.isel(south_north=slice(0,200),
             west_east=slice(0,200))

# assign projection and dim info
dt = dt.rio.write_crs(dt.attrs['pyproj_srs'])
dt_era5 = dt.rio.set_spatial_dims(x_dim='west_east',
                             y_dim='south_north')

# sum over time dimension
dt_era5_sum = dt_era5.sum(dim='time')

In [7]:
dt_era5_sum

<xarray.DataArray 'PRCP' (ens: 6, south_north: 200, west_east: 200)>
dask.array<sum-aggregate, shape=(6, 200, 200), dtype=float32, chunksize=(1, 200, 200), chunktype=numpy.ndarray>
Coordinates:
    lat          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
    lon          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
  * west_east    (west_east) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
  * south_north  (south_north) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
  * ens          (ens) object 'LIN_nourban_PRCP' ... 'WSM6_urban_PRCP'
    spatial_ref  int64 0

#### 1.1.3 Define Members

In [8]:
# ensemble member list
ensemble_members = list(dt.ens.values)

# urban and nourban members
urban_members = [ens for ens in ensemble_members if not 'nourban' in ens ]
nourban_members = [ens for ens in ensemble_members if 'nourban' in ens ]

#### 1.1.4 Ensemble Mean For Urban and Nourban

In [9]:
dt_era5_sum_urban_ens = dt_era5_sum.sel(ens=urban_members).mean(dim='ens')
dt_era5_urban_ens = dt_era5.sel(ens=urban_members).mean(dim='ens')

In [10]:
dt_era5_sum_urban_ens

<xarray.DataArray 'PRCP' (south_north: 200, west_east: 200)>
dask.array<mean_agg-aggregate, shape=(200, 200), dtype=float32, chunksize=(200, 200), chunktype=numpy.ndarray>
Coordinates:
    lat          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
    lon          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
  * west_east    (west_east) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
  * south_north  (south_north) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
    spatial_ref  int64 0

### 1.2 GFS

#### 1.2.1 Data Options

In [11]:
run_data = 'gfs' # gfs in normal

# data path
data_path = glob(fr'{general_path}/*{run_data}**{variable}*')

In [12]:
data_path

['data/gfs_LIN_nourban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/gfs_LIN_urban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/gfs_THOMPSON_nourban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/gfs_THOMPSON_urban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/gfs_WSM6_nourban_PRCP_wrfout_d03_2017-07-18.nc',
 'data/gfs_WSM6_urban_PRCP_wrfout_d03_2017-07-18.nc']

#### 1.1.2 Open Data

In [13]:
# open data
dt = xr.open_mfdataset(data_path,
                       concat_dim='ens')[variable]

# instead of 201 grid we want 200 grid in each direction
dt = dt.isel(south_north=slice(0,200),
             west_east=slice(0,200))

# assign projection and dim info
dt = dt.rio.write_crs(dt.attrs['pyproj_srs'])
dt_gfs = dt.rio.set_spatial_dims(x_dim='west_east',
                             y_dim='south_north')

# sum over time dimension
dt_gfs_sum = dt_gfs.sum(dim='time')

In [14]:
dt_gfs_sum

<xarray.DataArray 'PRCP' (ens: 6, south_north: 200, west_east: 200)>
dask.array<sum-aggregate, shape=(6, 200, 200), dtype=float32, chunksize=(1, 200, 200), chunktype=numpy.ndarray>
Coordinates:
    lat          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
    lon          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
  * west_east    (west_east) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
  * south_north  (south_north) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
  * ens          (ens) object 'LIN_nourban_PRCP' ... 'WSM6_urban_PRCP'
    spatial_ref  int64 0

#### 1.1.3 Ensemble Mean For Urban and Nourban

In [15]:
dt_gfs_sum_urban_ens = dt_gfs_sum.sel(ens=urban_members).mean(dim='ens')
dt_gfs_urban_ens = dt_gfs.sel(ens=urban_members).mean(dim='ens')

In [16]:
dt_gfs_sum_urban_ens

<xarray.DataArray 'PRCP' (south_north: 200, west_east: 200)>
dask.array<mean_agg-aggregate, shape=(200, 200), dtype=float32, chunksize=(200, 200), chunktype=numpy.ndarray>
Coordinates:
    lat          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
    lon          (south_north, west_east) float32 dask.array<chunksize=(200, 200), meta=np.ndarray>
  * west_east    (west_east) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
  * south_north  (south_north) float64 -1.09e+05 -1.08e+05 ... 8.9e+04 9e+04
    spatial_ref  int64 0

### 1.3 IMERG

In [47]:
# define general path to datasets
data_source = 'imerg'
imerg_var = 'IRprecipitation'
general_path = f'data/observation/{data_source}/*20170718*'

# get individual data links
data_links = glob(general_path)
imerg = xr.open_mfdataset(data_links)

# adjust imerg units and turn it from 30 mins interval to 1 hour sums
imerg = imerg.resample(time = '1H').sum()[imerg_var]
imerg = imerg.transpose(transpose_coords = ['lat', 'lon'])
imerg_sum = imerg.sum(dim='time')

imerg_sum = imerg_sum.rio.write_crs(CRS.from_epsg(4326))
imerg_sum = imerg_sum.rio.set_spatial_dims(x_dim='lon', y_dim='lat')

In [89]:
#print(da_to_be_matched.dims)
dt_match = dt_gfs_sum_urban_ens.copy(deep=True)
dt_match = dt_match.rio.set_spatial_dims(x_dim='west_east', y_dim='south_north')

imerg_sum_matched = imerg_sum.rio.reproject_match(dt_match, 
                                                 Resampling.cubic_spline
                                                 )

## 2. Intra-Simulation Metric Calculation

### 2.1 Concat All Ensemble Members

In [91]:
era5_gfs_concat = np.concatenate([dt_era5_sum.sel(ens=urban_members).values, 
               dt_gfs_sum.sel(ens=urban_members).values])

### 2.2 Concat Ensemble Mean

In [92]:
ens_era5_gfs_concat = np.concatenate([dt_era5_sum_urban_ens.values[np.newaxis],
                    dt_gfs_sum_urban_ens.values[np.newaxis]])

### 2.3 Concat All Simulations

In [93]:
simulations = np.concatenate([
    era5_gfs_concat,
    ens_era5_gfs_concat
]) 

### 2.4 Concat IMERG

In [94]:
imerg_and_simulations = np.concatenate([
    simulations,
    imerg_sum_matched.values[np.newaxis]
]) 

In [95]:
np.shape(imerg_and_simulations)

(9, 200, 200)

### 2.4 Calculate Metrics

In [96]:
# data used in simulation numpy array in order
simulation_data_order = [
    'era5-lin',
    'era5-thompson',
    'era5-wsm6',
    'gfs-lin',
    'gfs-thompson',
    'gfs-wsm6',
    'ens-era5',
    'ens-gfs',
    'imerg'
]

In [100]:
spaef = np.zeros((9,9), dtype=np.float64)
corr = np.zeros((9,9), dtype=np.float64)
cv_ratio = np.zeros((9,9), dtype=np.float64)
histo_match = np.zeros((9,9), dtype=np.float64)

for i, s1 in enumerate(imerg_and_simulations):
    
    print(fr'next first data is {simulation_data_order[i]}')
    for j, s2 in enumerate(imerg_and_simulations):
        
        print(fr'>> next second data is {simulation_data_order[j]}')
        
        # spaef calculation
        SPAef, alpha, beta, gamma = SPAEF(s1, s2)
        
        spaef[i, j] = SPAef
        corr[i, j] = alpha
        cv_ratio[i, j] = beta
        histo_match[i, j] = gamma
        
        
    print('END ------------------')

next first data is era5-lin
>> next second data is era5-lin
>> next second data is era5-thompson
>> next second data is era5-wsm6
>> next second data is gfs-lin
>> next second data is gfs-thompson
>> next second data is gfs-wsm6
>> next second data is ens-era5
>> next second data is ens-gfs
>> next second data is imerg
END ------------------
next first data is era5-thompson
>> next second data is era5-lin
>> next second data is era5-thompson
>> next second data is era5-wsm6
>> next second data is gfs-lin
>> next second data is gfs-thompson
>> next second data is gfs-wsm6
>> next second data is ens-era5
>> next second data is ens-gfs
>> next second data is imerg
END ------------------
next first data is era5-wsm6
>> next second data is era5-lin
>> next second data is era5-thompson
>> next second data is era5-wsm6
>> next second data is gfs-lin
>> next second data is gfs-thompson
>> next second data is gfs-wsm6
>> next second data is ens-era5
>> next second data is ens-gfs
>> next second

In [104]:
df_spaef = pd.DataFrame(spaef, columns = [simulation_data_order],
             index = simulation_data_order)
df_corr = pd.DataFrame(corr, columns = [simulation_data_order],
             index = simulation_data_order)
df_cv_ratio = pd.DataFrame(cv_ratio, columns = [simulation_data_order],
             index = simulation_data_order)
df_histo_match = pd.DataFrame(histo_match, columns = [simulation_data_order],
             index = simulation_data_order)

In [112]:
df_spaef

,era5-lin,era5-thompson,era5-wsm6,gfs-lin,gfs-thompson,gfs-wsm6,ens-era5,ens-gfs,imerg
era5-lin,1.000000,0.405058,0.688453,0.413470,0.349114,0.470367,0.737699,0.489903,0.062267
era5-thompson,0.400094,1.000000,0.266400,0.197037,0.294638,0.351379,0.429175,0.307943,-0.367692
era5-wsm6,0.688479,0.272420,1.000000,0.248763,0.218401,0.346818,0.709324,0.327769,0.155723
gfs-lin,0.398771,0.196744,0.233512,1.000000,0.708370,0.643436,0.210431,0.768703,-0.239854
gfs-thompson,0.329516,0.293592,0.197424,0.708290,1.000000,0.583002,0.160720,0.695891,-0.312629
gfs-wsm6,0.470150,0.352698,0.346322,0.654441,0.598650,1.000000,0.417556,0.856429,-0.153750
ens-era5,0.755991,0.485236,0.720351,0.280762,0.244266,0.435053,1.000000,0.425147,0.127049
ens-gfs,0.489517,0.308841,0.327056,0.782952,0.714390,0.856423,0.405556,1.000000,-0.131243
imerg,0.095020,-0.294554,0.185379,-0.116747,-0.174403,-0.112128,0.130393,-0.085096,1.000000


In [113]:
df_corr

,era5-lin,era5-thompson,era5-wsm6,gfs-lin,gfs-thompson,gfs-wsm6,ens-era5,ens-gfs,imerg
era5-lin,1.000000,0.460949,0.699213,0.460003,0.415989,0.475836,0.903590,0.499364,0.217557
era5-thompson,0.460949,1.000000,0.308989,0.224243,0.313363,0.395788,0.680665,0.342584,-0.140544
era5-wsm6,0.699213,0.308989,1.000000,0.291506,0.270883,0.358958,0.847547,0.339270,0.345925
gfs-lin,0.460003,0.224243,0.291506,1.000000,0.747317,0.709585,0.406848,0.912553,0.047701
gfs-thompson,0.415989,0.313363,0.270883,0.747317,1.000000,0.693445,0.408949,0.906291,0.024235
gfs-wsm6,0.475836,0.395788,0.358958,0.709585,0.693445,1.000000,0.501746,0.882870,-0.006985
ens-era5,0.903590,0.680665,0.847547,0.406848,0.408949,0.501746,1.000000,0.485478,0.197588
ens-gfs,0.499364,0.342584,0.339270,0.912553,0.906291,0.882870,0.485478,1.000000,0.024935
imerg,0.217557,-0.140544,0.345925,0.047701,0.024235,-0.006985,0.197588,0.024935,1.000000


In [114]:
df_cv_ratio

,era5-lin,era5-thompson,era5-wsm6,gfs-lin,gfs-thompson,gfs-wsm6,ens-era5,ens-gfs,imerg
era5-lin,1.000000,0.866389,1.020261,0.814497,0.791557,0.952571,1.180925,0.943485,1.361890
era5-thompson,1.154216,1.000000,1.177601,0.940105,0.913627,1.099472,1.363042,1.088985,1.571915
era5-wsm6,0.980141,0.849184,1.000000,0.798322,0.775837,0.933654,1.157473,0.924748,1.334844
gfs-lin,1.227752,1.063711,1.252628,1.000000,0.971835,1.169521,1.449883,1.158365,1.672063
gfs-thompson,1.263333,1.094538,1.288930,1.028981,1.000000,1.203415,1.491902,1.191936,1.720521
gfs-wsm6,1.049790,0.909527,1.071060,0.855051,0.830969,1.000000,1.239724,0.990461,1.429699
ens-era5,0.846794,0.733653,0.863951,0.689711,0.670285,0.806631,1.000000,0.798937,1.153240
ens-gfs,1.059901,0.918286,1.081375,0.863286,0.838972,1.009631,1.251663,1.000000,1.443468
imerg,0.734274,0.636167,0.749151,0.598064,0.581219,0.699448,0.867122,0.692776,1.000000


In [115]:
df_histo_match

,era5-lin,era5-thompson,era5-wsm6,gfs-lin,gfs-thompson,gfs-wsm6,ens-era5,ens-gfs,imerg
era5-lin,1.000000,0.786625,0.921400,0.865800,0.802175,0.940725,0.836375,0.920200,0.631000
era5-thompson,0.786625,1.000000,0.829325,0.801600,0.863600,0.786125,0.696575,0.802950,0.507400
era5-wsm6,0.921400,0.829325,1.000000,0.852625,0.829600,0.893650,0.809075,0.901675,0.584225
gfs-lin,0.865800,0.801600,0.852625,1.000000,0.857150,0.881425,0.736950,0.855875,0.577275
gfs-thompson,0.802175,0.863600,0.829600,0.857150,1.000000,0.803700,0.663725,0.783525,0.498275
gfs-wsm6,0.940725,0.786125,0.893650,0.881425,0.803700,1.000000,0.816925,0.917525,0.636025
ens-era5,0.836375,0.696575,0.809075,0.736950,0.663725,0.816925,1.000000,0.840950,0.692275
ens-gfs,0.920200,0.802950,0.901675,0.855875,0.783525,0.917525,0.840950,1.000000,0.636275
imerg,0.631000,0.507400,0.584225,0.577275,0.498275,0.636025,0.692275,0.636275,1.000000
